In [ ]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import os
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
import pickle
import gc

from PIL import Image

### Versão 1.0

In [ ]:
base_dir = '/content/gdrive/Shareddrives/TCC - S09/Notebooks/resized data/'

# with open(base_dir + 'BS_2011_2011-10-04_11-38-00_12Hz_2058_600.pkl', 'rb') as f:
#     data = pickle.load(f)

# with open(base_dir + 'YS_2017_2017-05-13_05-00-00_10Hz_2058_600.pkl', 'rb') as f:
#     data = pickle.load(f)

with open(base_dir + 'LJ_2018_2018-01-03_09-39-38_10Hz_2058_600.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
len(data)

1800

In [ ]:
# Split data into training/test set (80/20)
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_set, test_set = torch.utils.data.random_split(data, [train_size, test_size])

# Set the labels
label = 2
train_set = [(x, label) for x in train_set]
test_set = [(x, label) for x in test_set]

In [ ]:
idx_start = int(train_size/3)
idx_end = int(2*train_size/3)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_1.pkl', 'wb') as f:
    pickle.dump(train_set[:idx_start], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_2.pkl', 'wb') as f:
    pickle.dump(train_set[idx_start:idx_end], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_3.pkl', 'wb') as f:
    pickle.dump(train_set[idx_end:], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/val.pkl', 'wb') as f:
    pickle.dump(test_set, f)

In [ ]:
idx_start = int(train_size/3)
idx_end = int(2*train_size/3)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_1.pkl', 'rb') as f:
    train1 = pickle.load(f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_2.pkl', 'rb') as f:
    train2 = pickle.load(f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_3.pkl', 'rb') as f:
    train3 = pickle.load(f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/val.pkl', 'rb') as f:
    val = pickle.load(f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_1.pkl', 'wb') as f:
    pickle.dump(train1 + train_set[:idx_start], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_2.pkl', 'wb') as f:
    pickle.dump(train2 + train_set[idx_start:idx_end], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_3.pkl', 'wb') as f:
    pickle.dump(train3 + train_set[idx_end:], f)

with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/val.pkl', 'wb') as f:
    pickle.dump(val + test_set, f)

In [ ]:
with open('/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/train_1.pkl', 'rb') as f:
    tr = pickle.load(f)

len(tr)

959

In [ ]:
tr[400]


(<PIL.Image.Image image mode=RGB size=600x600 at 0x7F88229E0A90>, 1)

### Versão 2.0

In [ ]:
base_dir = '/content/gdrive/Shareddrives/TCC - S09/Notebooks/resized data/'

datasets = ['AA_2014_2014-03-27_09-10-00_12Hz_2058_600.pkl', 'YS_2017_2017-05-13_05-00-00_10Hz_2058_600.pkl', 'BS_2011_2011-10-01_16-18-00_15Hz_2058_600.pkl',
            'BS_2011_2011-10-04_11-38-00_12Hz_2058_600.pkl', 'BS_2011_2011-10-04_13-07-00_12Hz_2058_600.pkl', 'BS_2013_2013-09-25_12-15-01_12Hz_2058_600.pkl',
            'LJ_2018_2018-01-03_09-39-38_10Hz_2058_600.pkl']
labels = [1, 1, 0, 0, 0, 0, 2]

nb_train = 6
nb_val = 2

for i in range(len(datasets)):
  print(f'{datasets[i]} \n')
  with open(base_dir + datasets[i], 'rb') as f:
    data = pickle.load(f)

  # Split data into training/test set (80/20)
  train_size = int(0.8 * len(data))
  val_size = len(data) - train_size
  train_set, val_set = torch.utils.data.random_split(data, [train_size, val_size])

  del data
  gc.collect()

  # Set the labels
  label = labels[i]
  train_set = [(x, label) for x in train_set]
  val_set = [(x, label) for x in val_set]

  # Train set
  print('train \n')
  if nb_train != 1:
    step = int(train_size/nb_train)
    idx_start = 0
    idx_end = int(train_size/nb_train)

    for j in range(nb_train-1):
      print(f'j: {j}, idx_start: {idx_start}, idx_end: {idx_end} \n')
      if i == 0:
        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{j+1}.pkl', 'wb') as f:
          pickle.dump(train_set[idx_start:idx_end], f)
      
      else:
        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{j+1}.pkl', 'rb') as f:
          train = pickle.load(f)

        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{j+1}.pkl', 'wb') as f:
          pickle.dump(train + train_set[idx_start:idx_end], f)
      
        del train
        gc.collect()

      idx_start = idx_end
      idx_end += step


    # Last part
    print(f'j: last part, idx_start: {idx_start}, idx_end: {idx_end} \n')
    if i == 0:
        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{nb_train}.pkl', 'wb') as f:
          pickle.dump(train_set[idx_start:], f)

    else:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{nb_train}.pkl', 'rb') as f:
        last = pickle.load(f)

      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train_{nb_train}.pkl', 'wb') as f:
        pickle.dump(last + train_set[idx_start:], f)

      del last
      gc.collect()
  
  else:
    if i == 0:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train.pkl', 'wb') as f:
        pickle.dump(train_set, f)
    
    else:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train.pkl', 'rb') as f:
        train = pickle.load(f)

      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/train.pkl', 'wb') as f:
        pickle.dump(train + train_set, f)
      
      del train
      gc.collect()

  # Validation set
  print('validation \n')
  if nb_val != 1:
    step = int(val_size/nb_val)
    idx_start = 0
    idx_end = int(val_size/nb_val)

    
    for j in tqdm(range(nb_val-1)):
      print(f'j: {j}, idx_start: {idx_start}, idx_end: {idx_end} \n')
      if i == 0:
        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{j+1}.pkl', 'wb') as f:
          pickle.dump(val_set[idx_start:idx_end], f)
      
      else:
        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{j+1}.pkl', 'rb') as f:
          val = pickle.load(f)

        with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{j+1}.pkl', 'wb') as f:
          pickle.dump(val + val_set[idx_start:idx_end], f)
      
        del val
        gc.collect()

      idx_start = idx_end
      idx_end += step


    # Last part
    print(f'j: last part, idx_start: {idx_start}, idx_end: {idx_end} \n')
    if i == 0:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{nb_val}.pkl', 'wb') as f:
          pickle.dump(val_set[idx_start:], f)
    else:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{nb_val}.pkl', 'rb') as f:
        last = pickle.load(f)

      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_{nb_val}.pkl', 'wb') as f:
        pickle.dump(last + val_set[idx_start:], f)

      del last
      gc.collect()
  
  else:
    if i == 0:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val.pkl', 'wb') as f:
        pickle.dump(val_set, f)
    
    else:
      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val.pkl', 'rb') as f:
        val = pickle.load(f)

      with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val.pkl', 'wb') as f:
        pickle.dump(val + val_set, f)

      del val
      gc.collect()

AA_2014_2014-03-27_09-10-00_12Hz_2058_600.pkl 

train 

j: 0, idx_start: 0, idx_end: 479 

j: 1, idx_start: 479, idx_end: 958 

j: 2, idx_start: 958, idx_end: 1437 

j: 3, idx_start: 1437, idx_end: 1916 

j: 4, idx_start: 1916, idx_end: 2395 

j: last part, idx_start: 2395, idx_end: 2874 

validation 



  0%|          | 0/1 [00:00<?, ?it/s]

j: 0, idx_start: 0, idx_end: 360 



100%|██████████| 1/1 [00:04<00:00,  4.28s/it]


j: last part, idx_start: 360, idx_end: 720 

YS_2017_2017-05-13_05-00-00_10Hz_2058_600.pkl 

train 

j: 0, idx_start: 0, idx_end: 80 

j: 1, idx_start: 80, idx_end: 160 

j: 2, idx_start: 160, idx_end: 240 

j: 3, idx_start: 240, idx_end: 320 

j: 4, idx_start: 320, idx_end: 400 

j: last part, idx_start: 400, idx_end: 480 

validation 



...

j: last part, idx_start: 180, idx_end: 360 

LJ_2018_2018-01-03_09-39-38_10Hz_2058_600.pkl 

train 

j: 0, idx_start: 0, idx_end: 240 

j: 1, idx_start: 240, idx_end: 480 

j: 2, idx_start: 480, idx_end: 720 

j: 3, idx_start: 720, idx_end: 960 

j: 4, idx_start: 960, idx_end: 1200 

j: last part, idx_start: 1200, idx_end: 1440 

validation 



  0%|          | 0/1 [00:00<?, ?it/s]

j: 0, idx_start: 0, idx_end: 180 



100%|██████████| 1/1 [00:18<00:00, 18.25s/it]


j: last part, idx_start: 180, idx_end: 360 



In [ ]:
with open(f'/content/gdrive/Shareddrives/TCC - S09/Notebooks/batch data/leave out test/val_1.pkl', 'rb') as f:
  val = pickle.load(f)

len(val)

1152

In [ ]:
val[50]

(<PIL.Image.Image image mode=RGB size=600x600 at 0x7FABB9CC24D0>, 1)